# Chapter 4: Handling errors without exceptions

Exceptions and try/catch blocks break the referential transparency principle. Functional programming has a different way of dealing with errors and operations that may not yield a result due to a failure. `Either` and `Option` are the types used to handle such scenarios. They are both available in the Scala standard library.

Now, let's examine a situation that proves that try/catch breaks referential transparency.

#### The downsides of exceptions

Exceptions have two main downsides:
 - They break referential transparency
 - They are not type safe
 
 To better understand such downside, let's consider the following piece of code:

In [7]:
def sum(i: Int): Int = {
    val y: Int = throw new Exception("Fail")
    try {
        val x = 45 + 5
        x + y
    } catch { 
        case e: Exception => 43 
    }
}

defined function sum

The code above will terminate in an exception:

In [4]:
sum(1)

: 

This is where things get weird. By applying the substitution principle in the x + y expression, we will end up with a different result, because the referential transparency has been broken:

In [1]:
def sumInlined(i: Int): Int = {
    try {
        val x = 45 + 5
        x + ((throw new Exception("Fail")): Int) // the previous y has been inlined
    } catch { 
        case e: Exception => 43 
    }
}

defined function sumInlined

Even though all values are still the same, the result will be different:

In [2]:
sumInlined(1)

res1: Int = 43

Referentially transparent expressions do not depend on context and can be reasoned about locally. Consider the example below:

In [1]:
45 + 2

res0: Int = 47

#### Referentially transparent expressions can be reasoned locally, no need to understand context

The result will always be 47 for this expression, regardless of circumstances. Everything we need to know to figure out the result is in the expression itself, as opposed to the `throw` clause, we need not to understand the context around it to figure out the end result. In other words, depending on where the `throw` is placed, the result will be different, even though it is still the very same expression, therefore, context matters.

#### Exceptions break type safety
The signature of the function above does not tell that it may fail and interrupt execution. The compiler will also not force users to treat the error.

The last statement is not true for Java's checked exception, but they end up requiring a lot of boilerplate code (that is usually worked around by lazy programmers that declare empty `catch` blocks) and does not work for higher order functions. 

### Partial functions

Partial functions those which are not defined for all inputs. In OOP or Procedural, it's natural to handle these cases with a bunch of ifs for input corner cases that are impossible to calculate an output for. Other than ifs, such functions may throw exceptions for undefined outputs. Partial functions areone of the main uses of `IllegalArgumentsException`.

> A function may also be partial if it does not terminate (functions of return type `Nothing`?) for some cases (e.g.: an unrecoverable error). This kind of partiality is not discussed throughout the book.

Let's see an example:

In [1]:
def mean(xs: Seq[Double]): Double =
    if (xs.isEmpty)
        throw new ArithmeticException("mean of empty list")
    else xs.sum / xs.length

defined function mean

One possible alternative to deal with this scenario is to receive a default value that is returned when an empty sequence is provided. It reminds me of callbacks, the old Javascript ways of handling errors (ah, the callback hells). Let's see:

In [2]:
def meanWithDefault(xs: Seq[Double], ifEmpty: Double): Double =
    if (xs.isEmpty)
        ifEmpty
    else xs.sum / xs.length

defined function meanWithDefault

Not very good, callers can only return a double value, which is not very flexible and still error prone. 

### The Option type

The Option type allows an explicit representation of operations that may not always yield a value. It is represented by:

In [3]:
sealed trait Option[+A]
case class Some[+A](get: A) extends Option[A]
case object None extends Option[Nothing] 

defined trait Option
defined class Some
defined object None

The mean function can benefit from the Option type:

In [4]:
def mean(xs: Seq[Double]): Option[Double] =
    if (xs.isEmpty) None
    else Some(xs.sum / xs.length)

defined function mean

In [5]:
mean(Seq())

res4: Option[Double] = None

# Chapter 5: Strictness and laziness